In [ ]:
import pandas as pd
import geopandas as gpd
import os
import pathlib as pl
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties

In [ ]:
cwd = os.getcwd()
PRT_dir = pl.Path('..','PRT_Notebook').resolve()
print(cwd)
print(PRT_dir)

In [ ]:
combined_df = pd.read_csv(os.path.join(PRT_dir,'Summary_Stats.csv'))
combined_df

In [ ]:
# Make color ramp for charts
iter0ramp = ['#FAF5CD', '#FAF3B5', '#FAF19D', '#FBEF85', '#FBED6D', '#FCEB55', '#FCE93D', '#FDE725']
iter1ramp = ['#D1F5D3', '#C0EEC2', '#B0E8B2', '#9FE2A2', '#8FDB92', '#7ED582', '#6ECF72', '#5EC962']
iter2ramp = ['#D0F6F4', '#B7E7E5', '#9ED9D6', '#85CAC7', '#6CBCB8', '#53ADA9', '#3A9F9A', '#21918C']
iter3ramp = ['#C9D7F7', '#B4C4E7', '#A0B1D8', '#8C9EC8', '#778BB9', '#6378A9', '#4F659A', '#3B528B']
iter4ramp = ['#EFCBF7','#D6AEDF','#BE91C8','#A574B1','#8D5799','#743A82','#5C1D6B','#440154'] 
# iter1ramp = ['#F5E8C1', '#F1DEA5', '#EED589', '#EBCB6E', '#E7C252', '#E4B837', '#E1AF1B', '#DEA600']
# iter2ramp = ['#D5E7F2', '#B9D3E3', '#9DBFD5', '#81ABC7', '#6597B8', '#4983AA', '#2D6F9C', '#115C8E']
# iter3ramp = ['#D2F7E0', '#B7E2C7', '#9CCEAF', '#82BA97', '#67A67F', '#4D9267', '#327E4F', '#186A37']
# iter4ramp = ['#ffe3e8','#F2C2D9','#E6A2CB','#D981BD','#CD61AE','#C040A0','#B42092','#a80084'] 

In [ ]:
for p in range(10,100,10):

    fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9))
    plt.rcParams['font.family'] = 'Calibri'
    plt.subplots_adjust(wspace=0.15, hspace=0.3)
    ax = ax.flatten()
    
    for i, ax in zip(np.arange(1,13,1), ax):
        group_df = combined_df[combined_df['Group'] == i]
        group_df.reset_index(inplace=True, drop=True)
    
        prob_df = group_df[group_df['Prob'] == p].copy()
        
        # Filter to just certain columns
        iter123_mcc = prob_df[['Prob','Group','obs','pp','reals','iter','MCC']]
        # Make copy of each iteration, rename to Iteration number
        iter0_mcc = iter123_mcc[iter123_mcc['iter']==0].copy()
        iter0_mcc = iter0_mcc.drop(columns='iter')
        iter0_mcc.rename(columns={'MCC':'Iteration0'}, inplace=True)
        iter1_mcc = iter123_mcc[iter123_mcc['iter']==1].copy()
        iter1_mcc = iter1_mcc.drop(columns='iter')
        iter1_mcc.rename(columns={'MCC':'Iteration1'}, inplace=True)
        iter2_mcc = iter123_mcc[iter123_mcc['iter']==2].copy()
        iter2_mcc = iter2_mcc.drop(columns='iter')
        iter2_mcc.rename(columns={'MCC':'Iteration2'}, inplace=True)
        iter3_mcc = iter123_mcc[iter123_mcc['iter']==3].copy()
        iter3_mcc = iter3_mcc.drop(columns='iter')
        iter3_mcc.rename(columns={'MCC':'Iteration3'}, inplace=True)
        iter4_mcc = iter123_mcc[iter123_mcc['iter']==4].copy()
        iter4_mcc = iter4_mcc.drop(columns='iter')
        iter4_mcc.rename(columns={'MCC':'Iteration4'}, inplace=True)
        # Merge to 1 dataframe
        iter123_mcc = pd.merge(iter0_mcc, iter1_mcc[['reals','Iteration1']], on='reals', how='left')
        iter123_mcc = pd.merge(iter123_mcc, iter2_mcc[['reals','Iteration2']], on='reals', how='left')
        iter123_mcc = pd.merge(iter123_mcc, iter3_mcc[['reals','Iteration3']], on='reals', how='left')
        iter123_mcc = pd.merge(iter123_mcc, iter4_mcc[['reals','Iteration4']], on='reals', how='left')
        iter123_mcc = iter123_mcc.sort_values(by='reals')
        iter123_mcc.reset_index(inplace=True, drop=True)
        iter123_mcc['position'] = iter123_mcc.index+1
    
        # give colorramps to iter123_mcc
        iter123_mcc = iter123_mcc.sort_values(by='Iteration0')
        iter123_mcc['Iter0ramp'] = iter0ramp
        iter123_mcc = iter123_mcc.sort_values(by='Iteration1')
        iter123_mcc['Iter1ramp'] = iter1ramp
        iter123_mcc = iter123_mcc.sort_values(by='Iteration2')
        iter123_mcc['Iter2ramp'] = iter2ramp
        iter123_mcc = iter123_mcc.sort_values(by='Iteration3')
        iter123_mcc['Iter3ramp'] = iter3ramp
        iter123_mcc = iter123_mcc.sort_values(by='Iteration4')
        iter123_mcc['Iter4ramp'] = iter4ramp
        iter123_mcc = iter123_mcc.sort_index()
    
        Probability = iter123_mcc['Prob'][0]
        if iter123_mcc['obs'][0] == 'R25':
            Observations = 'Random 25'
        if iter123_mcc['obs'][0] == 'R100':
            Observations = 'Random 100'
        if iter123_mcc['obs'][0] == 'C25':
            Observations = 'Clustered 25'
        if iter123_mcc['obs'][0] == 'C100':
            Observations = 'Clustered 100'
            
        if iter123_mcc['pp'][0] == 'pp10':
            PilotPoints = 800
        if iter123_mcc['pp'][0] == 'pp25':
            PilotPoints = 128
        if iter123_mcc['pp'][0] == 'pp50':
            PilotPoints = 32
    
        # Make table for display below
        groupplot_df = iter123_mcc.copy()
        groupplot_df = groupplot_df[['reals','Iteration0','Iteration1','Iteration2','Iteration3','Iteration4']]
        groupplot_df.rename(columns={'reals':'Realizations', 'Iteration0':'Iteration 0', 'Iteration1':'Iteration 1', 'Iteration2':'Iteration 2', 'Iteration3':'Iteration 3','Iteration4':'Iteration 4'}, inplace=True)
        groupplot_df = groupplot_df.round(decimals=3)
        groupplot_df['Realizations'] = groupplot_df['Realizations'].astype(str)

        groupplot_df = groupplot_df.sort_values(by='Iteration 0')
        groupplot_df['Iter0ramp'] = iter0ramp
        groupplot_df = groupplot_df.sort_values(by='Iteration 1')
        groupplot_df['Iter1ramp'] = iter1ramp
        groupplot_df = groupplot_df.sort_values(by='Iteration 2')
        groupplot_df['Iter2ramp'] = iter2ramp
        groupplot_df = groupplot_df.sort_values(by='Iteration 3')
        groupplot_df['Iter3ramp'] = iter3ramp
        groupplot_df = groupplot_df.sort_values(by='Iteration 4')
        groupplot_df['Iter4ramp'] = iter4ramp
        groupplot_df = groupplot_df.sort_index()
        
        groupplot_arr = groupplot_df[['Iter0ramp','Iter1ramp','Iter2ramp','Iter3ramp','Iter4ramp']].to_numpy()
        # groupplot_arr = np.where(groupplot_arr >= 95, 'red','white')
        # tablecolors = [['white'] + groupplot_arr[0].tolist(), ['white'] + groupplot_arr[1].tolist(), ['white']+ groupplot_arr[2].tolist(),
        #               ['white']+ groupplot_arr[3].tolist(), ['white']+ groupplot_arr[4].tolist(), ['white']+ groupplot_arr[5].tolist(),
        #               ['white']+ groupplot_arr[6].tolist(), ['white']+ groupplot_arr[7].tolist()]
    
        # drop the color columns from grouplot_df
        groupplot_df = groupplot_df[['Realizations','Iteration 0','Iteration 1','Iteration 2', 'Iteration 3', 'Iteration 4']]
           
        # ax
        plt.rcParams['font.family'] = 'Calibri'
        ax.scatter(iter123_mcc['position']-0.3,iter123_mcc['Iteration0'], color=iter123_mcc['Iter0ramp'], edgecolors= "grey", s= 75, linewidth=1)
        ax.scatter(iter123_mcc['position']-0.15,iter123_mcc['Iteration1'], color=iter123_mcc['Iter1ramp'], edgecolors= "grey", s= 75, linewidth=1)
        ax.scatter(iter123_mcc['position'],iter123_mcc['Iteration2'], color=iter123_mcc['Iter2ramp'], edgecolors= "grey", s= 75, linewidth=1)
        ax.scatter(iter123_mcc['position']+0.15,iter123_mcc['Iteration3'], color=iter123_mcc['Iter3ramp'], edgecolors= "grey", s= 75, linewidth=1)
        ax.scatter(iter123_mcc['position']+0.3,iter123_mcc['Iteration4'], color=iter123_mcc['Iter4ramp'], edgecolors= "grey", s= 75, linewidth=1)
        ax.set_xticks([1,2,3,4,5,6,7,8],['10','25','50','100','250','500','1000','2000'])
        ax.set_yticks([0.4, 0.5,0.6, 0.7,0.8,0.9, 1])
        #ax.set_xlabel('Number of Realizations')
        ax.set_title(f"Pilot Points: {PilotPoints}", fontsize=12)
    
        if group_df.at[0,'Group'] == 1:
            ax.set_ylabel("Matthews Correlation Coefficent")
        if group_df.at[0,'Group'] == 5:
            ax.set_ylabel("Matthews Correlation Coefficent")
        if group_df.at[0,'Group'] == 9:
            ax.set_ylabel("Matthews Correlation Coefficent")
    
        if group_df.at[0,'Group'] == 9:
            ax.set_xlabel('Number of Realizations')
        if group_df.at[0,'Group'] == 10:
            ax.set_xlabel('Number of Realizations')
        if group_df.at[0,'Group'] == 11:
            ax.set_xlabel('Number of Realizations')
        if group_df.at[0,'Group'] == 12:
            ax.set_xlabel('Number of Realizations')
        
        # Get the x-tick locations
        xticks = ax.get_xticks()
        xticks = xticks +0.5
        xticks = xticks.tolist()
        xticks.pop()
        
        # Draw vertical lines between ticks
        for tick in xticks:
            ax.axvline(x=tick, color='gray', linestyle=(0, (1,1)), linewidth = 0.5)
    
    #fig.suptitle('Mathews Correlation Coefficient Score with Probability >50%', fontweight='bold', fontsize=20)
    plt.gcf().text(0.15, 0.91, 'Observations: Random 25', fontsize=12,fontweight='bold')
    plt.gcf().text(0.35, 0.91, 'Observations: Random 100', fontsize=12, fontweight='bold')
    plt.gcf().text(0.55, 0.91, 'Observations: Clustered 25', fontsize=12, fontweight='bold')
    plt.gcf().text(0.75, 0.91, 'Observations: Clustered 100', fontsize=12, fontweight='bold')
    
    # Legend Items
    
    iter0dot = Line2D([],[], color='#FDE725', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Prior')
    iter1dot = Line2D([],[], color='#5EC962', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 1')
    iter2dot = Line2D([],[], color='#21918C', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 2')
    iter3dot = Line2D([],[], color='#3B528B', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 3')
    iter4dot = Line2D([],[], color='#440154', marker='o', markeredgecolor='grey', linestyle='none', markersize=10, label='Posterior 4')
    
    LegendElement = [iter0dot, iter1dot, iter2dot, iter3dot, iter4dot]
    LegendElement = [i for i in LegendElement if i is not None]
    
    # legend
    fig.legend(handles=LegendElement, title=None, title_fontsize=13, loc='lower center', frameon=False, ncol=5,
              prop= {'size':12, 'family':'calibri'})# bbox_to_anchor=(-0.05,1.4))
    
    fig.savefig(f'MCC Prob{p}.png', bbox_inches='tight', dpi=300)